In [0]:
dbutils.widgets.text("outputfolder", "/dbfs/mnt/strukturparametre/forestblocks_planet_delineated/")
dbutils.widgets.text("output_filename", "forest.gpkg")

outfolder = dbutils.widgets.get("outputfolder")
print(outfolder)

In [0]:
#Merge forest blocks applying again buffer out, buffer in method to connect blocks at their borders of their max extent

import geopandas as gpd
import pandas as pd
import glob, os
from shapely.geometry import Point, Polygon
from shapely.geometry import shape
from shapely import geometry
import shutil





print('--------final merge-------------------')

def buffer_in_correction(bfr_in_input):
    
    bfrin_gdf = bfr_in_input.explode(index_parts=True)
    bfrin_gdf = gpd.GeoDataFrame(gpd.GeoSeries(bfrin_gdf))
    
    bfrin_gdf = bfrin_gdf.rename(columns={0:'geometry'}).set_geometry('geometry')
    

    
    
    bfrin_gdf = bfrin_gdf.set_crs(25832)
    return bfrin_gdf
def buffer_out_correction(bfr_out_output):
    bfrout_gdf = gpd.geoseries.GeoSeries([geom for geom in bfr_out_output.unary_union.geoms])
    bfrout_gdf = bfrout_gdf.set_crs(25832)
    return bfrout_gdf

def last_merge():
    
    clips_dir = '/dbfs/mnt/strukturparametre/forestblocks_planet_delineated/' #folder with 53 geopackages forest clips

    gpkg_pattern = os.path.join(clips_dir, '*.gpkg')
    file_list = glob.glob(gpkg_pattern)

     #merge geopackages as geodataframes
        
    dfs = []
    for file in file_list:
        forest=gpd.read_file(file)
        gdf_forest = gpd.GeoDataFrame(forest)

        dfs.append(gdf_forest)
        
    df = pd.concat(dfs)  #merged geodataframes in data frame
    
    gdf = gpd.GeoDataFrame(df)

    gdf = gdf.to_crs(epsg=25832)

   

    buffered = gdf.to_crs(25832).buffer(10)
    print('Buffer out')

    buffered = buffer_out_correction(buffered)

    buffered = buffered.buffer(-10,resolution=2 )
    print('buffer in')
    
    
    buffered =buffer_in_correction(buffered)
    
    buffered['area_ha']= buffered.area/10000

    forest = buffered[buffered['area_ha'] >= 0.5]
    print('small areas')
    
    print(len(forest))
    forest.to_file("/tmp/Planet_Forest_Denmark_2020.gpkg", layer='Planet_Forest_Denmark_2020', driver="GPKG")
    shutil.move("/tmp/Planet_Forest_Denmark_2020.gpkg", "/dbfs/mnt/strukturparametre/Planet_Forest_Denmark_2020.gpkg" )
 #   forest.to_file("/dbfs/mnt/strukturparametre/Forest_Denmark_2020_test.gpkg", layer='Forest_Denmark_2020', driver="GPKG")
    
    

last_merge()



--------final merge-------------------
Buffer out